In [ ]:
from sklearn.model_selection import train_test_split
from IPython.display import Image
from sklearn import tree
from os import system
import networkx as nx 
import pandas as pd
import collections
import numpy as np
import pydotplus
import re
import os

In [ ]:
def proc_value_v2(lable_str):
    value = [int(re.search(r'value = \[(\d+), (\d+)\]', lable_str).group(x)) for x in range(1,3)]
    prc = max(value) / sum(value) * 100
    return prc


def max_value_from_label(lable_str):    
    return max([int(re.search(r'value = \[(\d+), (\d+)\]', lable_str).group(x)) for x in range(1,3)])


def get_name_for_node(G):
    name_nodes = dict.fromkeys(list(G.nodes))
    for x in G.nodes:
        list_neighbors = [x for x in nx.all_neighbors(G, x)]
        if x == '0':
            name_nodes[x] = 'Root'
            name_nodes[list_neighbors[0]] = 'True'
            name_nodes[list_neighbors[1]] = 'False'
        elif len(list_neighbors) < 2:
            # Leaf
            pass
        elif len(list_neighbors) == 3:
            name_nodes[list_neighbors[1]] = 'True'
            name_nodes[list_neighbors[2]] = 'False'
        else:
            print('What do u do in this case?!')
    return name_nodes


def sort_char_like_num(list_char):
    list_num = [map(int, list_char)]
    list_num.sort()
    return [map(str, list_num)]


def change_relational_op(relational):
    relational = relational.split(' ')
    if relational[1] == '<=':
        return relational[0]+' > ' + relational[2], True
    elif relational[1] == '=>':
        return relational[0]+' < ' + relational[2], True
    else:
        return relational, False

In [ ]:
FIlE_PATH = 'DATA/'
FILE_PATH_RESULT = 'DATA/RESULT/'
FILE_NAME = '202006_data_for_tree.csv'
PNG_NAME = 'dtree.png'
DOT_NAME = 'tree.dot'
MAX_DEPTH = 5
df = pd.read_csv(FIlE_PATH + FILE_NAME, index_col=0)
print(len(df))
df = df.fillna(0)

In [ ]:
train, test = train_test_split(df.drop(columns =['customer_id','count_f','allow_f']), 
                               test_size=0.1,
                               random_state=8,
                               shuffle=True)
X_n_train = train.drop(columns = ['target']).copy()
Y_n_train = train['target'].copy()
X_n_test = test.drop(columns = ['target']).copy()
Y_n_test = test['target'].copy()

## Generate many trees
#### and save all of them into dictionary

In [ ]:
dict_rule = {}
TARGET = 'class = F'

for rand in range(1,10):    
    for depth in range(3,6):
        dectree = tree.DecisionTreeClassifier(random_state=rand, max_depth=depth, criterion='entropy')
        dectree.fit(X_n_train, Y_n_train)
        
        dot_data = tree.export_graphviz(dectree,
                             out_file=None, 
                             feature_names = X_n_train.columns, 
                             class_names = ['NOT_F','F'],
                             filled=True,
                             rounded=True)

        # Save tree (graph) in pydotplus format
        pydotplus_graph = pydotplus.graph_from_dot_data(dot_data)
        # Save tree (graph) in networkx format
        G = nx.nx_pydot.from_pydot(pydotplus_graph)
        
        # Find leafs in Tree
        leaf_nodes = [x for x in G.nodes() if G.out_degree(x)==0 and G.in_degree(x)==1]
        name_nodes = get_name_for_node(G)

        # Create list of target leafs
        target_leaf = [x for x in leaf_nodes if G.nodes[x]['label'].find(TARGET) != -1 
                       and proc_value_v2(G.nodes[x]['label']) > 90]

        # Create list of target nodes (the path from root to target leaf)
        target_nodes = list()
        list_path_to_target = []
        for x in target_leaf:
            path_to_target = list(nx.all_simple_paths(G, '0', x))[0]
            list_path_to_target.append(path_to_target)
            target_nodes += path_to_target

        # Save trees labels into dict
        for path in list_path_to_target:
            rule = ''
            key_rule = ''
            # print(path)
            rule += str(path) +'\n'
            for node in range(len(path)):
                label = pydotplus_graph.get_node(path[node])[0].get_attributes()['label']
                if len(label.split('\\n')) > 4:
                    way = name_nodes[path[node+1]]
                    relational = label.split('\\n')[0]
                    if way == 'False':
                        relational, tmp = change_relational_op(relational)
                        if tmp:
                            rule += 'True ' + relational + '\n'
                        else:
                            rule += way + ' ' + relational + '\n'
                    else:
                        rule += way + ' ' + relational + '\n'
                else:
                    max_value = str(max_value_from_label(label))
                    print_line = TARGET + ' - value: ' + max_value + ' ' + str(proc_value_v2(label)) + '%'
                    rule += print_line
            # key_rule = str(depth) + '_' + str(rand) + '_' + path[-1] + '_' + max_value
            key_rule = str(depth) + '_' + path[-1] + '_' + max_value
            dict_rule[key_rule] = rule

print(len(dict_rule))

In [ ]:
len(dict_rule)

## View Condition Trees

In [ ]:
TOP_NUM = 3
sorted_trees = sorted(list(dict_rule.keys()), key=lambda d: int(d.split('_')[-1]), reverse=True)
for key in sorted_trees[:TOP_NUM]:
    print(key, '\n', dict_rule[key], '\n')

## Save tree in png file

In [ ]:
top_depth = int(sorted_trees[0].split('_')[0])

# Create Tree
dectree = tree.DecisionTreeClassifier(random_state=0, max_depth=top_depth, criterion='entropy')
dectree.fit(X_n_train, Y_n_train)

# Save Tree in dot format
dot_data = tree.export_graphviz(dectree,
                     out_file=None, 
                     feature_names = X_n_train.columns, 
                     class_names = ['NOT_F','F'],
                     filled=True,
                     rounded=True)

# Save in pydotplus format
pydotplus_graph = pydotplus.graph_from_dot_data(dot_data)
# Save in networkx format
G = nx.nx_pydot.from_pydot(pydotplus_graph)

# Find leafs in Tree
leaf_nodes = [x for x in G.nodes() if G.out_degree(x)==0 and G.in_degree(x)==1]
name_nodes = get_name_for_node(G)

# Append info into dot file: 
#     1) Node number in lablel 

for node in pydotplus_graph.get_node_list()[2:]:
    node_num = node.to_string().split(' ')[0]
    name_nodes
    node.set('label', 'Node=' + node_num + '\\n' 
             + node.get_attributes()['label'][1:])

#     2) Headlabel in edge
for edge in pydotplus_graph.get_edge_list():
    bolean_name = name_nodes[edge.get_destination()]
    edge.set('headlabel', '"' + bolean_name + '"')
    edge.set('labeldistance', '2.5')
    if bolean_name == 'True':
        edge.set('labelangle', '45')
    else:
        edge.set('labelangle', '-45')

# Create list of target leafs
target_leaf = [x for x in leaf_nodes if G.nodes[x]['label'].find(TARGET) != -1 
               and proc_value_v2(G.nodes[x]['label']) > 90]

# Create list of target nodes (the path from root to target leaf)
target_nodes = list()
list_path_to_target = []
for x in target_leaf:
    path_to_target = list(nx.all_simple_paths(G, '0', x))[0]
    list_path_to_target.append(path_to_target)
    target_nodes += path_to_target

# Del not result Nodes
for x in G.nodes:
    if x not in set(target_nodes):
        pydotplus_graph.del_node(x)

# Save into PNG format 
pydotplus_graph.write_png(FILE_PATH_RESULT + 'graph_V3.png')
# Display Tree
Image(FILE_PATH_RESULT + 'graph_V3.png')